# 1D Wave Propagation through a Heterogeneous Domain

In this notebook, we no longer solve domains monolithically (single time step), we introduce the concept of multi time
stepping with an algorithm that calls two instances of the Simple Integrator for two individually solved subdomains

## Multi-Time Step Integration 
This module describes a multi-time step integration algorithm for two subdomains through enforcing
continuity of acceleration across the interface of subdomains

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import imageio 
import sys
import os
sys.path.append('..')
from proposed.SimpleIntegrator import SimpleIntegrator
from boundaryConditions.BoundaryConditions import VelBoundaryConditions as vbc
from boundaryConditions.BoundaryConditions import AccelBoundaryConditions as abc


## Create a Multi-Time Step Class

We create a module for Integrating with Multiple Time Steps
It contains the following methods:
 - Calculation of Time Step Ratios for large $\Omega_L$ and small $\Omega_s$ subdomains
    - Current Time Step Ratio $ t^{n+k}_{ratio} = \frac{t_s^{n+k} - t_L^N}{t_{TL}^{N+1} - t_L^N}$

    - Next Time Step Ratio $ t^{n+k+1}_{ratio} = \frac{(t_s^{n+k} + \Delta t_s^{n+k}) - t_L^N}{t_{TL}^{N+1} - t_L^N}$

    - where Trial Time $ t_{Ti} = t_i + \Delta t_{Ci} $
    
 - Integration Communication between both subdomains with the summation and prescription of system variables
    - Mass Summation $ \mathbf{M}_\Gamma = (\mathbf{C}^T_s \mathbf{M}_s \mathbf{C}_s) + (\mathbf{C}^T_L \mathbf{M}_L \mathbf{C}_L) $

    - Internal Force Summation $ \mathbf{f}^{int}_\Gamma = \mathbf{C}_s^T \mathbf{f}^{int}_s + \mathbf{C}_L^T \mathbf{f}^{int}_L $

    - Continuity of Acceleration $ \mathbf{\ddot{u}}_{\Gamma} = \mathbf{\ddot{u}}_{\Gamma s} = \mathbf{\ddot{u}}_{\Gamma L} $

 - Calculation of Reduction Factors Quantities
    - Large Reduction Factor $ \alpha_L = 1-\frac{(t_{TL}^N - t_s^{n+k})}{(t_{TL}^{N+1}-t_L^N)}$

    - Small Reduction Factor $ \alpha_s = 1-\frac{(t_{Ts}^{n+k} - t_{TL}^{N+1})}{(t_{Ts}^{n+k+1}-t_s^{n+k})}$

 

In [28]:
class Proposed_MTS:

    """
    Constructor for the subcycling class
    It accepts two domains a large and a small one
    They are both SimpleIntegrators, and they ratio is a non-integer number:
    LARGE     |   SMALL
    *----*----*--*--*--*--*
    """
    def __init__(self, largeDomain: SimpleIntegrator, smallDomain: SimpleIntegrator):
        
        self.large = largeDomain
        self.small = smallDomain
        # Interface
        self.mass_Gamma = self.large.mass[-1] + self.small.mass[0]
        self.f_int_Gamma = self.large.f_int[-1] + self.small.f_int[0]
        # Time step Ratio Computations
        self.large_tTrial = 0.0
        self.small_tTrial = 0.0
        self.nextTimeStepRatio = 0.0 
        self.currTimeStepRatio = 0.0
        # Reduction Factor Values
        self.large_alpha = 0.0
        self.small_alpha = 0.0
        # Tolerance
        self.tol = 1e-6

    def calc_timestep_ratios(self):
        self.small_tTrial = self.small.t + self.small.dt
        self.large_tTrial = self.large.t + self.large.dt
        self.currTimeStepRatio = (self.small.t - self.large.t) / (self.large_tTrial - self.large.t)
        self.nextTimeStepRatio = ((self.small.t + self.small.dt) - self.large.t) / (self.large_tTrial - self.large.t)

    def accelCoupling(self): 
        return -self.f_int_Gamma / self.mass_Gamma

    def update_small_domain(self):
        self.small.a_bc.indexes.append(0)
        self.small.a_bc.accelerations.append(self.accelCoupling)
        self.small.single_tstep_integrate()
        self.small.assemble_internal()
        self.calc_timestep_ratios()

    def integrate(self):    
        if ((self.large.t == 0) and (self.small.t == 0)):
            self.small.assemble_internal()
            self.large.assemble_internal()
            self.f_int_Gamma = self.large.f_int[-1] + self.small.f_int[0]
            self.calc_timestep_ratios()

        while (self.nextTimeStepRatio <= 1 or (self.currTimeStepRatio <= 1 and self.nextTimeStepRatio <= 1 + self.tol)):
            # Integrate Small Domain
            self.update_small_domain()

        # Compute Reduction Factors 
        self.alpha_L = 1 - ((self.large_tTrial - self.small.t)/(self.large_tTrial - self.large.t))
        self.alpha_s = 1 - ((self.small_tTrial - self.large_tTrial)/(self.small_tTrial - self.small.t))

        if (self.alpha_L >= self.alpha_s):
            self.large.dt = self.alpha_L * self.large.dt
        elif (self.alpha_s > self.alpha_L):
            self.small.dt = self.alpha_s * self.small.dt
            self.update_small_domain()
            
        # Integrate Large Domain
        self.large.a_bc.indexes.append(-1)
        self.large.a_bc.accelerations.append(self.accelCoupling)
        self.large.single_tstep_integrate()
        self.large.assemble_internal()        
        self.calc_timestep_ratios()

        # Interface Internal Force Summation
        self.f_int_Gamma = self.large.f_int[-1] + self.small.f_int[0] 

    

## Output Plots for Multi Time Step Class

A Class is created to plot the FEM 1D Problems for the Heterogeneous Domain Case
This includes the creation of a plot for:
 - Velocity throughout the domain (.gif)

In [29]:
class Visualise_MultiTimestep:

    def __init__(self, fullDomain: Proposed_MTS):

        self.total = fullDomain
        self.filenames = []

    def plot(self):
        self.filenames.append(f'FEM1D{self.total.large.n}.png')
        plt.style.use('ggplot')
        plt.plot(self.total.large.position, self.total.large.v, "--")
        plt.plot(self.total.small.position + self.total.large.L , self.total.small.v, "--")
        plt.title(f"1D Wave Propagation through Heterogeneous Media", fontsize=9)
        plt.xlabel("Domain Position (mm)", fontsize=9)
        plt.ylabel("Velocity (mm/ms)", fontsize=9)
        plt.legend([f"Large Time Step Domain", "Small Time Step Domain"])
        plt.savefig(f'FEM1D{self.total.large.n}.png')
        plt.close()

    def create_gif(self):
        with imageio.get_writer('Notebook_Multi-time-step.gif', mode='I') as writer:
            for filename in self.filenames:
                image = imageio.imread(filename)
                writer.append_data(image)

        for filename in set(self.filenames):
            os.remove(filename)

## 1D Heterogeneous Domain Example Domain

In this numerical example we simulate the propagation of a square wave through a
Heterogeneous elastic domain. Due to the difference in dilatational wave speeds, one notices that
the wave travels through at different speeds with reflection as well at the interface

Suppose a domain $\Omega$ is split into two subdomains $\Omega_L$ and $\Omega_s$ of similar discretisations, with simple elastic isotropic properties of $E_L= 0.02$ GPa, $E_s= \rho(\frac{\pi}{0.02})^2$ GPa and $\rho_L=\rho_s = 8000 $ kgm $^{-3}$. 
A prescribed velocity boundary condition is applied to the large domain at $x=0$ with an equivalent sine period of $L_L/2 \cdot \sqrt{\rho/E_L}$.

In [30]:
def proposedCoupling():
    # Utilise same element size, drive time step ratio with Material Properties
    nElemLarge = 300
    E_L = 0.02e9
    rho = 8000
    E_s = (np.pi/0.02)**2 * rho # Non Integer Time Step Ratio = pi
    Courant = 0.5
    Length = 50e-3
    propTime = 1.75 * Length * np.sqrt(rho / E_L)    
    def vel(t): return vbc.velbcSquare(t, 2 * Length , E_L, rho)
    accelBCs_L = abc(list(),list())
    accelBCs_s = abc(list(),list())
    propTime = 1 * Length * np.sqrt(rho / E_L) 

    # Initialise with default material properties 
    young_L = np.full(nElemLarge, E_L)
    density_L = np.full(nElemLarge, rho)
    young_S = np.full(2* nElemLarge, E_s)
    density_S = np.full(2* nElemLarge, rho)

    Domain_L = SimpleIntegrator("total",young_L, density_L, Length, 1, 
                                       nElemLarge, propTime, vbc([0], [vel]), accelBCs_L, Courant)
    Domain_S = SimpleIntegrator("total", young_S, density_S, 2* Length, 1, 
                                       2* nElemLarge, propTime, None, accelBCs_s, Courant)
    full_Domain = Proposed_MTS(Domain_L, Domain_S)

    plotfullDomain = Visualise_MultiTimestep(full_Domain)
    # Solve Loop
    while(full_Domain.large.t <= 0.0016):
        full_Domain.integrate()
        if (full_Domain.large.n % 20 == 0):
            plotfullDomain.plot()
    plotfullDomain.create_gif()

if __name__ == "__main__":
    proposedCoupling()